# Plotting Figure 1

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import simulation
from analysis import * 

In [ ]:
def readDataFigure1(Round):
    """
    Attribute=round: which is the max Round number you want to read. Corresponding to the different pannels
    The data has to be loaded localy.
    
    One can change the generation over which is calculated using:
    startIndex, endIndex = (70000,generations)
    """

    generations = 100000
    numberOfRounds = Round
    groupSize = 2
    popSize = 100
    riskInRound = simulation.RiskInRound(1)
    numberOfGames = 1000
    wealthPoor = 1
    wealthRich = 1
    heterogeneous = 0
    
    filename = "figure1_V1_"
    
    arraytypeOfRiskCurve = np.array([1,2,3,1])
    arrayglobalLambdaValue = np.array([1,10,7,10])
    arrayAlphaRich = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
    arrayAlpahPoor = arrayAlphaRich
    
    ContributionFig1Round1 = np.empty(shape=(4,len(arrayAlphaRich)))# [typeOfRiskCurve,alpha's,]
    
    startIndex, endIndex = (70000,generations)
    
    for row, (typeOfRiskCurveNumber,globalLambdaValue) in enumerate(zip(arraytypeOfRiskCurve,arrayglobalLambdaValue)):
        for column, (alphaRich,alphaPoor) in enumerate(zip(arrayAlphaRich,arrayAlpahPoor)):
            typeOfRiskCurve = simulation.RiskCurve(typeOfRiskCurveNumber)
            #### Load the file and the header #####
            StringName = "%s_%d_%d_%d_%d_%s_%0.2f_%0.2f_%d_%d_%d_%s_%d_%0.2f.dat" % (filename, generations, numberOfRounds, groupSize, popSize, riskInRound.name, alphaPoor, alphaRich, numberOfGames, wealthPoor, wealthRich, typeOfRiskCurve.name, heterogeneous, globalLambdaValue)
            
            # Have IsError because I forgot some files. To detect that just skip them. 
            IsError=False
            try:
                HeaderFile = readHeader(filename=StringName)
            except:
                IsError=True
                
            if not IsError:
                #### Compute the averaging over the relevant itterations #####
                # Uncomment the method that you want to use:
            
                # Read out summary
                #summary = readSummary(filename=StringName,heterogeneous=HeaderFile["heterogenous"])
                #roundContribution =  summary['AverageContributionPerRoundRich']
            
                # Read out just the generations from start to end index
                ContributionArray = readContributionFileToData(fileName = StringName, heterogeneous = HeaderFile["heterogenous"], rounds = int(HeaderFile["number_of_rounds"]))
                roundContribution = np.mean(ContributionArray[startIndex:endIndex,:],axis=0)
                
                #### Save the contribution levels in the tensor #####
                
                ContributionFig1Round1[row,column] = np.sum(roundContribution)
            else:
                ContributionFig1Round1[row,column] = 0
                
            
    
    return ContributionFig1Round1

Data1Round1 = readDataFigure1(Round=1)
Data1Round2 = readDataFigure1(Round=2)
Data1Round4 = readDataFigure1(Round=4)

In [ ]:
AllData = np.empty(shape=(4,4,10))
AllData[0]=Data1Round1
AllData[1]=Data1Round2
AllData[2]=Data1Round4

In [ ]:
# Define the diferent risk curves.
# here we assume that W_0 = 1 such that it all fits on the plot. 

def linear(Contribution,lambda_):
    probLoss = (1.0 - (Contribution)*lambda_)
    
    probLoss[probLoss<0]=0
    return probLoss

def Power(Contribution,lambda_):
    probLoss = (1.0 - (Contribution) ** lambda_)
    
    probLoss[probLoss<0]=0
    return probLoss

def StepWise(Contribution,lambda_):
    probLoss = 1.0 / ( np.exp(lambda_*(Contribution - 0.5)) + 1.0 )
    
    probLoss[probLoss<0]=0
    return probLoss

In [ ]:
# Generate the curves:
x= np.linspace(0,1,100)
linear1 = linear(x,1)
linear2 = linear(x,10)
power = Power(x,10)
stepWise = StepWise(x,7)
RiskCurves = [linear1,power,stepWise,linear2]

In [ ]:
def plotFigure1(Data):
    """
    Plot figure 1
    Provided data is a nested list with the following substructure:
    [round,typeRiskCurve,alpha]
    """
    fig, axs = plt.subplots(nrows=2, ncols=2,figsize=(15,10))
    
    alpha = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
    label = ["linear1","PowerLaw","StepFunction","linear2"]
    Round = [1,2,4]
    colors = ["tab:orange","k","tab:blue","tab:red"]
    markers = ["o","s","^","D"]
    figure = ["a)","b)","c)","d)"]
    
    # Plot the data
    for i in range(1,4):
        row = i//2
        column =i%2
        ax = axs[row][column]
        for j in range(4):
            ax.plot(alpha,Data[i-1,j],label=label[j],color=colors[j],marker=markers[j])   
        #ax.legend()
        ax.set_xlabel("Loss fraction \u03B1",fontsize=15)
        ax.set_ylabel("Contribution",fontsize=15)
        ax.set_title(figure[i]+"  "+f"\u03A9 = {Round[i-1]}",fontsize=18)
        ax.tick_params(axis='both', which="both",direction ="in",bottom=True, top=True, left=True, right=True, labelsize=14)
        ax.set_ylim(-0.05, 1)
    
    # Plot the risk curves
    ax_risk = axs[0][0]
    for i in range(4):
        ax_risk.plot(x,RiskCurves[i],color=colors[i])
    ax_risk.set_ylim(-0.05, 1.05)
    ax_risk.set_xlabel("Contribution",fontsize=15)
    ax_risk.tick_params(axis='both', which="both",direction ="in",bottom=True, top=True, left=True, right=True, labelsize=14)
    ax_risk.set_ylabel("Risk probability",fontsize=15)
    ax_risk.set_title(figure[0]+"  ""Risk curves",fontsize=18)
    
    # adjust spacing inbetween the subplots:
    plt.subplots_adjust( wspace=None, hspace=0.3)
    
    plt.savefig("Figure1.png")
    
    
plotFigure1(AllData)